----------MODEL TO PREDICT THE POPULARITY OF A SONG-----------

In [24]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
import math

In [2]:
train = pd.read_csv("C:\\Users\\Shaurya_nsut\\Documents\\B Tech\\Societies\\GDSC\\Recruit\\Round 2\\data\\tracks.csv")
artists = pd.read_csv("C:\\Users\\Shaurya_nsut\\Documents\\B Tech\\Societies\\GDSC\\Recruit\\Round 2\\data\\artists.csv")
display(train.head(5))
display(train.tail(5))

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.7440,0.151,0.127,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.0000,0.148,0.655,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.0218,0.212,0.457,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.9180,0.104,0.397,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.1300,0.311,0.196,103.220,4


,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.518,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.663,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.314,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.615,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4
586671,5Ocn6dZ3BJFPWh4ylwFXtn,Mar de Emociones,38,214360,0,['Afrosound'],['0i4Qda0k4nf7jnNHmSNpYv'],2015-07-01,0.686,0.723,6,-7.067,1,0.0363,0.105,0.000000,0.2640,0.9750,112.204,4


In [3]:
#ID is not needed for our model.
#Similarly ID of artist is also not needed.
#Name is also not needed but artist name does affect the model
print(train.nunique())
#But more than 100k artist names which makes it impractical to use them in the model
#Still keep because we will need it later in the code
#OneHotEncoding and LabelEncoding both will not be ideal to encode such huge amount of data

id                  586672
name                446474
popularity             101
duration_ms         123122
explicit                 2
artists             114030
id_artists          115062
release_date         19700
danceability          1285
energy                2571
key                     12
loudness             29196
mode                     2
speechiness           1655
acousticness          5217
instrumentalness      5402
liveness              1782
valence               1805
tempo               122706
time_signature           5
dtype: int64


In [4]:
#Removing useless columns
train = train.drop(['id','name','id_artists','artists'], axis = 1)
print(train.columns)

Index(['popularity', 'duration_ms', 'explicit', 'release_date', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature'],
      dtype='object')


In [5]:
#Calculating NaN values per column for imputation
train.isnull().sum()

popularity          0
duration_ms         0
explicit            0
release_date        0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
dtype: int64

Since we have 0 NaN values in each column, we can entirely skip the imputation part otherwise we would have had to use imputation functions like SimpleImputer() provided by scikit learn.

In [6]:
#Checking for non-numeric data
train.select_dtypes(include = 'object').columns

Index(['release_date'], dtype='object')

'release_date' is a non-numeric datatype in our dataframe. Best method would be to classify songs based on how old they are. So we create an entirely new feature, 'age' where we classify this datavfrom very old to very recent and then use OneHotEncoding to encode the categorical data.

In [7]:
train['release_date'].head(20)

0     1922-02-22
1     1922-06-01
2     1922-03-21
3     1922-03-21
4           1922
5           1922
6           1922
7           1922
8           1922
9     1922-03-29
10    1922-06-01
11    1922-06-01
12    1922-02-22
13          1922
14          1922
15    1922-06-01
16    1922-06-01
17    1922-06-01
18          1922
19    1922-03-21
Name: release_date, dtype: object

In many cases only year is given and not the whole date, so we should convert all dates to years because it is more helpful for us.

In [8]:
train['release_year'] = train['release_date'].str[:4].astype(int)
train['release_year']

0         1922
1         1922
2         1922
3         1922
4         1922
          ... 
586667    2020
586668    2020
586669    2020
586670    2021
586671    2015
Name: release_year, Length: 586672, dtype: int64

In [9]:
#Creating new feature, 'age'
df1 = pd.DataFrame(train['release_year'])
bins = [1920,1950,1970,1980,1990,2000,2010,2025]
labels = ['Early20th_century','Mid20th_century','very_old','old','recent','new','very_new']
train['age'] = pd.cut(train['release_year'], bins=bins, labels=labels, right=False)
train.columns

Index(['popularity', 'duration_ms', 'explicit', 'release_date', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'release_year', 'age'],
      dtype='object')

In [10]:
#Dropping useless columns
train = train.drop(['release_date','release_year'], axis = 1)
train.columns

Index(['popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'age'],
      dtype='object')

In [11]:
#Encoding the age column using pd.get_dummies function
train_enc = pd.get_dummies(train, columns = ['age'])
train_enc.head(5)

,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,...,valence,tempo,time_signature,age_Early20th_century,age_Mid20th_century,age_very_old,age_old,age_recent,age_new,age_very_new
0,6,126903,0,0.645,0.4450,0,-13.338,1,0.4510,0.674,...,0.127,104.851,3,True,False,False,False,False,False,False
1,0,98200,0,0.695,0.2630,0,-22.136,1,0.9570,0.797,...,0.655,102.009,1,True,False,False,False,False,False,False
2,0,181640,0,0.434,0.1770,1,-21.180,1,0.0512,0.994,...,0.457,130.418,5,True,False,False,False,False,False,False
3,0,176907,0,0.321,0.0946,7,-27.961,1,0.0504,0.995,...,0.397,169.980,3,True,False,False,False,False,False,False
4,0,163080,0,0.402,0.1580,3,-16.900,0,0.0390,0.989,...,0.196,103.220,4,True,False,False,False,False,False,False


In [12]:
train_enc.columns

Index(['popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'age_Early20th_century', 'age_Mid20th_century', 'age_very_old',
       'age_old', 'age_recent', 'age_new', 'age_very_new'],
      dtype='object')

In [13]:
#dividing into data and target variable
X_train = train_enc.drop(['popularity'],axis=1)
y_train = train_enc['popularity']

In [14]:
trainX = pd.DataFrame(StandardScaler().fit_transform(X_train), columns = X_train.columns)

In [15]:
display(trainX)

,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,...,valence,tempo,time_signature,age_Early20th_century,age_Mid20th_century,age_very_old,age_old,age_recent,age_new,age_very_new
0,-0.815233,-0.214754,0.490096,-0.385182,-1.483654,-0.615393,0.719665,1.924128,0.642528,2.362779,...,-1.650527,-0.457392,-1.845842,3.763482,-0.404917,-0.343264,-0.40401,-0.477356,-0.416822,-0.521566
1,-1.042088,-0.214754,0.791115,-1.107625,-1.483654,-2.344110,0.719665,4.736917,0.995129,-0.425120,...,0.398600,-0.552876,-6.072724,3.763482,-0.404917,-0.343264,-0.40401,-0.477356,-0.416822,-0.521566
2,-0.382618,-0.214754,-0.780204,-1.449000,-1.199517,-2.156266,0.719665,-0.298309,1.559864,-0.343432,...,-0.369823,0.401596,2.381040,3.763482,-0.404917,-0.343264,-0.40401,-0.477356,-0.416822,-0.521566
3,-0.420026,-0.214754,-1.460507,-1.776084,0.505310,-3.488663,0.719665,-0.302756,1.562730,3.014787,...,-0.602678,1.730782,-1.845842,3.763482,-0.404917,-0.343264,-0.40401,-0.477356,-0.416822,-0.521566
4,-0.529308,-0.214754,-0.972856,-1.524420,-0.631241,-1.315289,-1.389536,-0.366127,1.545530,0.062013,...,-1.382743,-0.512190,0.267599,3.763482,-0.404917,-0.343264,-0.40401,-0.477356,-0.416822,-0.521566
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,0.223004,-0.214754,-0.021636,-0.095410,-1.483654,0.537413,-1.389536,-0.420604,0.960729,-0.425120,...,-1.324530,0.451253,0.267599,-0.265711,-0.404917,-0.343264,-0.40401,-0.477356,-0.416822,1.917304
586668,-0.606659,-0.214754,1.212541,0.480163,-1.483654,0.979122,0.719665,-0.220485,-0.885409,-0.424007,...,0.518909,1.062561,0.267599,-0.265711,-0.404917,-0.343264,-0.40401,-0.477356,-0.416822,1.917304
586669,-0.335506,-0.214754,-0.172146,-0.905182,0.505310,-0.514201,-1.389536,-0.356121,1.276063,-0.424558,...,-1.886099,0.894707,0.267599,-0.265711,-0.404917,-0.343264,-0.40401,-0.477356,-0.416822,1.917304
586670,-0.695890,-0.214754,0.797135,0.289629,1.357723,0.784793,0.719665,-0.391142,-0.699075,-0.425111,...,-0.443560,-0.955375,0.267599,-0.265711,-0.404917,-0.343264,-0.40401,-0.477356,-0.416822,1.917304


In [16]:
display(y_train)

0          6
1          0
2          0
3          0
4          0
          ..
586667    50
586668    72
586669    70
586670    58
586671    38
Name: popularity, Length: 586672, dtype: int64

In [17]:
#performing train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainX, y_train, test_size=0.2, random_state=42)

In [28]:
#Making model
model =XGBRegressor(
        objective="reg:squarederror",  
        n_estimators=1000, 
        booster = 'gbtree',
        learning_rate=0.01,
        max_depth = 12,
        eval_metric="rmse",
        random_state=42,
        n_jobs=-1,
        verbosity=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [29]:
#calculating root mean square value
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)
print("RMSE is: ", rmse)

RMSE is:  13.014201696638224
